In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../model")
from MLP_classifier import MultiClassClassifier
from dataset import OOD, DeepFakeDatasetFastLoad
import torch
from torch.utils.data import DataLoader
sys.path.append("../tools")
from constants import INT_TO_GEN_DATA3, GEN_TO_INT_DATA3

/home/lsaland/micromamba/envs/clip/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ood = OOD("../../data/ood.pt",load_preprocessed=True)

In [4]:
device = "cuda"
model = MultiClassClassifier()
model.load_state_dict(torch.load("../model/checkpoints/multiclass_1000epochs_0.08loss.pt"))
model.eval()
model.to(device)
model.set_generators_maps(gen_to_int=GEN_TO_INT_DATA3,int_to_gen=INT_TO_GEN_DATA3)

In [26]:
d = {}
for gen in ood.int_to_gen:
    d[ood.int_to_gen[gen]] = {"features":ood.features[ood.gen == gen], 
                              'label':ood.label[ood.gen == gen]}

In [35]:
pred = {}
for gen in d:
    pred[gen] = torch.mean(torch.eq(d[gen]["label"],model.predict_binary(d[gen]["features"],device=device).cpu()).float()).item()

In [36]:
pred

{'null': 0.8299999833106995,
 'Lexica': 0.699999988079071,
 'Ideogram': 0.7699999809265137,
 'Leonardo': 0.75,
 'Copilot': 0.949999988079071,
 'img2img_SD1.5': 0.15000000596046448,
 'Photoshop_generativemagnification': 0.14000000059604645,
 'Photoshop_generativefill': 0.20000000298023224}